Implement Stacking ensemble method in the questions no. 3 and train the meta learner for prediction. 

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, f1_score
from sklearn.svm import SVC
from sklearn.naive_bayes import MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier

In [2]:
def letsgo(file_path):
    data = pd.read_csv(file_path)   #reading csv file
    data = pd.DataFrame(data)        #inserting data into data frame
    a = 0.3                  #proportion of test data 
    b = 18                  #random state
    
    x = data.iloc[:,range(data.shape[1]-1)]      #x is data containing all rows and all columns except for quality
    xCopy = x.copy()
    y = data.iloc[:,data.shape[1]-1]             #y is data containing all rows and one column with quality of wine
    
    x = (x - np.mean(x, axis = 0))/np.std(x)     #normalizing data
    
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = a, random_state = b)   #train test split
    
    #logistic regression with maximum iteration of 10000 for multinomial and tolerance of 1*10**(-15) 
    lr = LogisticRegression(max_iter = 10000, multi_class= "multinomial", solver= "saga", tol = 1e-15)
    lr.fit(x_train,y_train)
    report = lr.predict(x_test)
    
    #support vector with biase of 3 and tolerance of 1*10**(-15)
    svc = SVC(C= 3, kernel="rbf", tol= 1e-10)
    svc.fit(x_train,y_train)
    report1 = svc.predict(x_test)
    
    #using non normalized data on naive bayes classifier due to issue with negative data
    x_train2, x_test2, y_train2, y_test2 = train_test_split(xCopy, y, test_size = a, random_state = b)
    nv = MultinomialNB()
    nv.fit(x_train2,y_train2)
    report2 = nv.predict(x_test2)
    
    #knn classifier with l1 distance calculator
    knn = KNeighborsClassifier(p = 1)
    knn.fit(x_train,y_train)
    report3 = knn.predict(x_test)
    
    #data frame containing predicted value of all above classifier
    df = pd.DataFrame({'Logistic': report, 'SVC': report1, 'Naive Bayes': report2,  "KNN":report3})

    return df, y_test

In [3]:
def model(file_path):
    output = letsgo(file_path)  #getting tuple of dataframe containing predictions of different models and y_test data
    tree = DecisionTreeClassifier()
    tree.fit(output[0],output[1])    #fit the dataframe containing predictions of different models and y_test data
    report4 = tree.predict(output[0])   #getting prediction after ensemble stacking method
    print("\n\nDecission Tree Classifier:\n\n")
    print("Classification Report on test data:\n",classification_report(output[1], report4, zero_division = 0),"\n\nAccuracy:\n",accuracy_score(output[1], report4))
    print("-------------------------------------------------------------------------------------------")   

In [4]:
model(file_path = "E:\Milan\MDS\machine learning\Dataset\wine+quality\winequality-red.csv")



Decission Tree Classifier:


Classification Report on test data:
               precision    recall  f1-score   support

           3       1.00      0.33      0.50         3
           4       0.50      0.06      0.11        17
           5       0.63      0.82      0.72       195
           6       0.66      0.61      0.64       199
           7       0.75      0.47      0.58        64
           8       0.00      0.00      0.00         2

    accuracy                           0.65       480
   macro avg       0.59      0.38      0.42       480
weighted avg       0.66      0.65      0.64       480
 

Accuracy:
 0.6541666666666667
-------------------------------------------------------------------------------------------


E:\anacondaMDS\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)


In [5]:
file_path = "E:\Milan\MDS\machine learning\Dataset\wine+quality\winequality-white.csv"
model(file_path)



Decission Tree Classifier:


Classification Report on test data:
               precision    recall  f1-score   support

           3       0.60      0.38      0.46         8
           4       0.80      0.21      0.33        38
           5       0.67      0.61      0.64       425
           6       0.62      0.78      0.69       683
           7       0.62      0.45      0.52       265
           8       0.69      0.22      0.33        50
           9       0.00      0.00      0.00         1

    accuracy                           0.63      1470
   macro avg       0.57      0.38      0.43      1470
weighted avg       0.64      0.63      0.62      1470
 

Accuracy:
 0.6346938775510204
-------------------------------------------------------------------------------------------


E:\anacondaMDS\lib\site-packages\sklearn\neighbors\_classification.py:228: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  mode, _ = stats.mode(_y[neigh_ind, k], axis=1)
